In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/cloudpickle.py:365: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if obj.__module__ is "__builtin__":


In [4]:
spark = SparkSession.builder.appName('cruproj').getOrCreate()

In [5]:
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [9]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [10]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol="cruiselineIndex")

In [11]:
indexed = indexer.fit(data).transform(data)
indexed.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiselineIndex=16.0)]

In [12]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruiselineIndex']

In [13]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruiselineIndex'],
                           outputCol='features')

In [14]:
output = assembler.transform(indexed)

In [15]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruiselineIndex: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiselineIndex=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))]

In [17]:
final_data = output.select('features','crew')

In [18]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [19]:
train, test = final_data.randomSplit([0.7,0.3])

In [20]:
train.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              116|
|   mean|7.860258620689665|
| stddev|3.529025351642253|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [21]:
test.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               42|
|   mean|7.611666666666666|
| stddev|3.467499919402582|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [22]:
from pyspark.ml.regression import LinearRegression

In [23]:
lr = LinearRegression(labelCol='crew')

In [24]:
lr_model = lr.fit(train)

In [25]:
test_results = lr_model.evaluate(test)

In [26]:
test_results.rootMeanSquaredError

0.7484452261376617